<a href="https://colab.research.google.com/github/AlisonJD/tb_examples/blob/main/Dynamic_plots_in_a_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic plots in a web app

##Using Tinybird, Streamlit and ngrok

###Example of New York City Yellow Taxi Trips

In [1]:
!pip install -q streamlit
!pip install pyngrok

     |████████████████████████████████| 8.3 MB 4.5 MB/s 
     |████████████████████████████████| 4.3 MB 11.3 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
     |████████████████████████████████| 111 kB 59.4 MB/s 
     |████████████████████████████████| 180 kB 53.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 124 kB 56.5 MB/s 
     |████████████████████████████████| 788 kB 56.7 MB/s 
     |████████████████████████████████| 370 kB 62.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatibl

Restart runtime

In [1]:
import streamlit as st
from pyngrok import ngrok

In [2]:
#!ngrok authtoken <your token from https://dashboard.ngrok.com/get-started/your-authtoken>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


##Node in Tinybird 

```
%
SELECT 
  toHour(tpep_pickup_datetime) as hr, 
  count() as trips 
FROM node_0
WHERE toDate(tpep_pickup_datetime) 
BETWEEN {{Date(start_date, '2019-01-01')}} AND {{Date(end_date, '2019-01-31')}}
AND puborough_id = {{UInt8(pick_up_id, 0)}} AND doborough_id = {{UInt8(drop_off_id, 0)}} 
GROUP BY hr
```

## Streamlit app

In [35]:
%%writefile app.py
import streamlit as st
import datetime

import pandas as pd
import plotly.express as px

def main():
  st.title("New York City Yellow Taxi Trips")

  st.sidebar.title("Select boroughs and dates")

  boroughs = ['Bronx', 'Brooklyn', 'EWR', 'Manhattan','Queens','Staten Island']
  pick_up = st.sidebar.radio('Pick-up borough', boroughs)
  drop_off = st.sidebar.radio('Drop-off borough', boroughs)

  pick_up_id=boroughs.index(pick_up)
  drop_off_id=boroughs.index(drop_off)

  start_date = st.sidebar.date_input('Start date', min_value = datetime.date(2019,1,1), max_value = datetime.date(2019,12,31), value = datetime.date(2019, 1, 1))
  end_date = st.sidebar.date_input('End date', min_value = datetime.date(2019,1,1), max_value = datetime.date(2019,12,31), value = datetime.date(2019, 1, 31))

  start = start_date.strftime("%Y-%m-%d")
  end = end_date.strftime("%Y-%m-%d")

  text = "From " + pick_up + " to " + drop_off
  st.subheader(text)

  text = "in the period " + start + " to " + end
  st.subheader(text)

  output = 'https://api.tinybird.co/v0/pipes/taxi_boroughs_by_hr.csv?start_date='+start+'&end_date='+end+'&pick_up_id='+str(pick_up_id)+'&drop_off_id='+str(drop_off_id)+'&token=p.eyJ1IjogImNkMDZkYzdlLTA5NWEtNDA0YS1iODNkLWQ1NzUwNmViYWZlZCIsICJpZCI6ICJhYjE5YzU1Ny1mYzlkLTQ4MmUtOTY0NC00ZjNlYmIxNWE3ZjIifQ.xbE9Ht1ZxKUrcG2HpHclTBOjctlYSPUm_GYzlehow6o'
  df=pd.read_csv(output)
  
  plot = px.bar(
                data_frame=df,
                x = "hr",
                y = "trips",
                color = "percent",
                title="How trips vary over the day",
                labels={
                     "hr": "Hour of the day",
                     "trips": "Total number of trips"
                 }
            )
  st.plotly_chart(plot, use_container_width=True)

  text = "Compare this to total New York City trips over the same period"
  st.subheader(text)

  output = 'https://api.tinybird.co/v0/pipes/taxi_by_hr.csv?start_date='+start+'&end_date='+end+'&token=p.eyJ1IjogImNkMDZkYzdlLTA5NWEtNDA0YS1iODNkLWQ1NzUwNmViYWZlZCIsICJpZCI6ICIzMmVmYTZjMy1iY2Q4LTQxYTMtOTVkMy1iNWYyMDEyZmM5MmEifQ.6SsAU9SOROgPJN9jaZACb9Xqn-EKTL602xK0t0p2j8o'
  df=pd.read_csv(output)
  
  plot = px.bar(
                data_frame=df,
                x = "hr",
                y = "trips",
                color = "percent",
                title="How trips vary over the day",
                labels={
                     "hr": "Hour of the day",
                     "trips": "Total number of trips"
                 }
            )
  st.plotly_chart(plot, use_container_width=True)

  st.subheader("Note how Manhattan dominates the trips!")

  st.markdown('#### Example Tinybird node')

  st.markdown('''
  %

SELECT 

  toHour(tpep_pickup_datetime) as hr, 

  count() as trips 

FROM node_0

WHERE toDate(tpep_pickup_datetime) 

BETWEEN {{Date(start_date, '2019-01-01')}} AND {{Date(end_date, '2019-12-31')}}

AND puborough_id = {{UInt8(pick_up_id, 0)}} AND doborough_id = {{UInt8(drop_off_id, 0)}} 

GROUP BY hr

'''
)

if __name__ == '__main__':
	main()

Overwriting app.py


In [37]:
!streamlit run --server.port 80 app.py &>/dev/null&
url=ngrok.connect()
url

2021-10-17 18:32:10.603 INFO    pyngrok.ngrok: Opening tunnel named: http-80-0c1e6a14-8594-465d-9777-756b66e31042
2021-10-17 18:32:10.638 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="no configuration paths supplied"
2021-10-17 18:32:10.641 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2021-10-17 18:32:10.643 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2021-10-17 18:32:10.653 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-10-17 18:32:10.820 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2021-10-17 18:32:10.822 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg="client session established" obj=csess id=d695c8d31b4f
2

<NgrokTunnel: "http://bc99-35-185-43-242.ngrok.io" -> "http://localhost:80">

2021-10-17 18:32:10.935 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg=start pg="/api/tunnels/http-80-0c1e6a14-8594-465d-9777-756b66e31042 (http)" id=3742cad3b113be10
2021-10-17 18:32:10.940 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:10+0000 lvl=info msg=end pg="/api/tunnels/http-80-0c1e6a14-8594-465d-9777-756b66e31042 (http)" id=3742cad3b113be10 status=200 dur=217.222µs


In [36]:
!killall ngrok

2021-10-17 18:32:07.941 INFO    pyngrok.process.ngrok: t=2021-10-17T18:32:07+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
